In [2]:
from pymongo import MongoClient

In [6]:
import pickle

In [11]:
from pprint import pprint

In [3]:
client = MongoClient('mongodb://localhost:27017/')

In [4]:
client.list_database_names()

['admin', 'config', 'local']

In [5]:
vac_db = client.vacancies_db

In [7]:
with open('vacancies', 'rb') as f:
    vacancies = pickle.load(f)

In [9]:
vac_db.rabota_ru.insert_many(vacancies)

In [12]:
for vacancy in vac_db.rabota_ru.find():
    pprint(vacancy)

{'_id': ObjectId('63275289df6222ca60758ff5'),
 'platform': 'nn.rabota.ru',
 'ref': 'https://nn.rabota.ru/vacancy/vacancy/35485411/?search_id=1663520004703349itfwf78r',
 'salary': {'comment': None,
            'currency': 'руб.',
            'max': '45000',
            'min': '35000'},
 'salary_raw': '35\xa0000\xa0— 45\xa0000 руб.',
 'title': 'Бухгалтер'}
{'_id': ObjectId('63275289df6222ca60758ff6'),
 'platform': 'nn.rabota.ru',
 'ref': 'https://nn.rabota.ru/vacancy/vacancy/46942440/?search_id=1663520004703349itfwf78r',
 'salary': {'comment': 'От', 'currency': None, 'max': None, 'min': '78850'},
 'salary_raw': 'от 78\xa0850 руб.',
 'title': 'Главный бухгалтер'}
{'_id': ObjectId('63275289df6222ca60758ff7'),
 'platform': 'nn.rabota.ru',
 'ref': 'https://nn.rabota.ru/vacancy/vacancy/46869275/?search_id=1663520004703349itfwf78r',
 'salary': {'comment': None,
            'currency': 'руб.',
            'max': '40000',
            'min': '30000'},
 'salary_raw': '30\xa0000\xa0— 40\xa0000 руб.

In [ ]:
def uniqueness_vacancy_check(vacancy_one, vacancy_two):
    pass